# Model Training
- Author: Myles Dunlap


This notebook is used to train a model using a single configuration file. The steps in this notebook are used in the Python script.

# User Inputs

In [1]:
# Append the path for the custom modules
path_custom_modules = '../'

# Path to the YAML config. file
path_cfg = {'base_dir': '../cfgs',
            'filename': 'train-0.yaml'}

# Import Libraries and Modules

In [2]:
# Libraries
import sys
import os
from pathlib import Path

import argparse
import inspect

# Append Path to Custom Modules
sys.path.append(path_custom_modules)

# Custom Modules
from src.models import llm_multiclass
from src.utils import (RecursiveNamespace,
                       seed_everything,
                       load_cfg,
                       RunIDs)
from src.dataloading.load_data import LoadData
from src.dataloading.stratify import StratifyData
from src.dataloading.preprocess import PreprocessData

# Allow HF tokenizer parallelism
os.environ['TOKENIZERS_PARALLELISM'] = 'True'

In [3]:
# Load the configuration file
CFG = load_cfg(base_dir=Path(path_cfg['base_dir']),
               filename=path_cfg['filename'])

# Set random seed on everything
seed_everything(seed=CFG.seed)

In [4]:
# Group ID and ID for each fold tested
run_ids = RunIDs(test_folds=CFG.cv.val_folds,
                 num_folds=CFG.cv.num_folds)
run_ids.generate_run_ids()

# Print the group id and ids for each fold
print(f'Group ID: {run_ids.group_id}')
for fold_num in CFG.cv.val_folds:
    fold_id = getattr(run_ids.folds_id,
                      f'fold{fold_num}').run_id
    entire_id = f'{run_ids.group_id}-{fold_id}' 
    print((f'ID for Testing Fold #{fold_num}: '
           f'{fold_id}\n\tFull/Entire ID: {entire_id}'))

Group ID: e0fde485
ID for Testing Fold #1: 2acbab2e
	Full/Entire ID: e0fde485-2acbab2e
ID for Testing Fold #2: 96333769
	Full/Entire ID: e0fde485-96333769
ID for Testing Fold #3: b0e81d31
	Full/Entire ID: e0fde485-b0e81d31
ID for Testing Fold #4: 2f30fa69
	Full/Entire ID: e0fde485-2f30fa69
ID for Testing Fold #5: 0f4a10d7
	Full/Entire ID: e0fde485-0f4a10d7


In [5]:
# Load Data from Disk
load_data_file = LoadData(base_dir=CFG.paths.data.base_dir)
if CFG.debug:
    data = load_data_file.load(filename=CFG.paths.data.debug_data)
else:
    data = load_data_file.load(filename=CFG.paths.data.data)


In [6]:
# Stratify the Data
data = (StratifyData(technique=CFG.stratify.technique,
                     n_folds=CFG.cv.num_folds,
                     target=CFG.data_info.target)
            .stratify(df=data))
cols = CFG.data_info.source_fields + \
       [CFG.data_info.target, 'fold']
display(data[cols].head(5))
print(f'Distribution of a Product for Each Fold')
print(f'Notice how the quantities are evenly distributed across folds')
display(data.groupby('Product').fold.value_counts()
        .sort_index().head(5))

,Consumer complaint narrative,State,Company response to consumer,Product,fold
0,The Summer of XX/XX/2018 I was denied a mortga...,IL,Closed with explanation,"Credit reporting, credit repair services, or o...",1
1,There are many mistakes appear in my report wi...,VA,Closed with explanation,"Credit reporting, credit repair services, or o...",1
2,There are many mistakes appear in my report wi...,TX,Closed with explanation,"Credit reporting, credit repair services, or o...",1
3,There are many mistakes appear in my report wi...,TX,Closed with explanation,"Credit reporting, credit repair services, or o...",1
4,There are many mistakes appear in my report wi...,CA,Closed with explanation,"Credit reporting, credit repair services, or o...",1


Distribution of a Product for Each Fold
Notice how the quantities are evenly distributed across folds


Product                  fold
Bank account or service  1       2953
                         2       2953
                         3       2954
                         4       2953
                         5       2953
Name: count, dtype: int64

In [7]:
from transformers import AutoTokenizer, DataCollatorWithPadding
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(f'{CFG.model_tokenizer.base_dir}/'
                                          f'{CFG.model_tokenizer.name}')

# Collator
collator = DataCollatorWithPadding(tokenizer=tokenizer,
                                   return_tensors='pt')

Start looping over folds here

In [8]:
from torch.utils.data import DataLoader
# Train a model for each validation fold
fold_num = CFG.cv.val_folds[0]

# Split Data into Training and Validation
df_train = data.copy()[data.fold != fold_num].reset_index(drop=True)
df_val = data.copy()[data.fold == fold_num].reset_index(drop=True)
print(f'Train Number of Instances: {len(df_train):,}')
print(f'Validation Number of Instances: {len(df_val):,}')
# Preprocessing





Train Number of Instances: 305,739
Validation Number of Instances: 76,435


## Preprocessing

Notes:
- Preprocessing techniques should be fitted ONLY on TRAIN data. Then those fits will be applied to the VALIDATION data. Do not fit on both TRAIN and VALIDATION (this an cause data leakage).
- The field `Company response to consumer` is a string and this will be combined with the unstructured text field `Consumer complaint narrative`.
    - This step will be performed in the dataset class.

In [9]:
a=CFG.preprocessing.apply_techniques[0]
getattr(CFG.preprocessing, a).fields

['State']

In [10]:
# Preprocessing Encoders
encoders = {}
for technique in CFG.preprocessing.apply_techniques:
    fields = getattr(CFG.preprocessing, technique).fields
    for col in fields:
        enc = PreprocessData(y=df_train[col].values,
                            technique=technique)
        encoders[col] = {'encoder': enc.encoder,
                         'technique': technique}
        print(col)


State


ValueError: Expected 2D array, got 1D array instead:
array=['Checking or savings account' 'Checking or savings account'
 'Checking or savings account' ... 'Payday loan' 'Credit reporting'
 'Mortgage'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
a = encoders['Product']['encoder']


In [ ]:
# Train and Validation Dataloaders

# # Train a model for each validation fold
# for fold_num in CFG.stratify.val_folds:
#     print(f'Starting Training for Fold {fold_num}')
    
#     # Training Module
    
#     # Inference Module
    
#     print(f'\tFinished Training for Fold {fold_num}')
# print('Training and Validation Completed')
